In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import IsolationForest
from joblib import dump
from sklearn.preprocessing import StandardScaler
from time import time

In [4]:
traffic_data = pd.read_csv('traffic_data_ALL_features.csv')
traffic_data['datetime'] = pd.to_datetime(traffic_data['datetime'], format='%Y-%m-%d %H:%M:%S')
traffic_data

,datetime,grid_id,estimated_traffic,G1辅路,G4辅路,G6辅路,一经路,七星路,万丰路,万华街,...,is_weekend,quarter,day_of_month,week_of_year,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos
0,2008-02-02 13:30:00,10_1,65,0,0,0,0,0,0,0,...,1,1,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958
1,2008-02-02 13:30:00,10_10,33995,0,0,1,0,0,0,0,...,1,1,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958
2,2008-02-02 13:30:00,10_11,2730,0,0,1,0,0,0,0,...,1,1,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958
3,2008-02-02 13:30:00,10_12,780,0,0,0,0,0,0,0,...,1,1,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958
4,2008-02-02 13:30:00,10_13,260,0,0,0,0,0,0,0,...,1,1,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173451,2008-02-08 17:30:00,5_13,195,0,0,0,0,0,0,0,...,0,1,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649
173452,2008-02-08 17:30:00,19_19,260,0,0,0,0,0,0,0,...,0,1,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649
173453,2008-02-08 17:30:00,19_20,650,0,0,0,0,0,0,0,...,0,1,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649
173454,2008-02-08 17:30:00,3_14,0,0,0,0,0,0,0,0,...,0,1,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649


In [5]:
traffic_data_if = traffic_data[['estimated_traffic', 'grid_id', 'l1', 'l2', 'l3', 'l4', 'l5', 'lag_24h', 'lag_48h', "datetime",
       'lag_72h', 'hour', 'day_of_week', 'is_weekend', 'quarter',
       'day_of_month', 'week_of_year', 'hour_sin', 'hour_cos',
       'day_of_week_sin', 'day_of_week_cos', 'day_of_month_sin',
       'day_of_month_cos']]
traffic_data_if.columns

Index(['estimated_traffic', 'grid_id', 'l1', 'l2', 'l3', 'l4', 'l5', 'lag_24h',
       'lag_48h', 'datetime', 'lag_72h', 'hour', 'day_of_week', 'is_weekend',
       'quarter', 'day_of_month', 'week_of_year', 'hour_sin', 'hour_cos',
       'day_of_week_sin', 'day_of_week_cos', 'day_of_month_sin',
       'day_of_month_cos'],
      dtype='object')

In [6]:
traffic_data_if

,estimated_traffic,grid_id,l1,l2,l3,l4,l5,lag_24h,lag_48h,datetime,...,is_weekend,quarter,day_of_month,week_of_year,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos
0,65,10_1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,1,1,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958
1,33995,10_10,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,1,1,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958
2,2730,10_11,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,1,1,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958
3,780,10_12,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,1,1,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958
4,260,10_13,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,1,1,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173451,195,5_13,130.0,65.0,65.0,195.0,0.0,455.0,130.0,2008-02-08 17:30:00,...,0,1,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649
173452,260,19_19,585.0,455.0,520.0,650.0,585.0,585.0,585.0,2008-02-08 17:30:00,...,0,1,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649
173453,650,19_20,780.0,780.0,780.0,650.0,780.0,910.0,975.0,2008-02-08 17:30:00,...,0,1,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649
173454,0,3_14,0.0,0.0,0.0,0.0,65.0,0.0,0.0,2008-02-08 17:30:00,...,0,1,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649


In [60]:
df_isolation = traffic_data_if.copy()

# ISOLATION TREE

In [ ]:
features = ['estimated_traffic', 'grid_id', 'l1', 'l2', 'l3', 'l4', 'l5', 'lag_24h', 'lag_48h',
       'lag_72h', 'hour', 'day_of_week', 'is_weekend', 'quarter',
       'day_of_month', 'week_of_year', 'hour_sin', 'hour_cos',
       'day_of_week_sin', 'day_of_week_cos', 'day_of_month_sin',
       'day_of_month_cos',
]

X = df_isolation[features]

preprocessor = ColumnTransformer([
    ('grid', OneHotEncoder(handle_unknown='ignore'), ['grid_id'])
], remainder='passthrough')

# Transform features
X_processed = preprocessor.fit_transform(X)

start = time()

# Train Isolation Forest model
model = IsolationForest(
    contamination=0.05,
    random_state=42,
    n_estimators=100,
    verbose=1
)
model.fit(X_processed)

train_time = time() - start
with open('isolation_forest_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Add predictions to dataframe (optional)
df_isolation['anomaly_score'] = model.decision_function(X_processed)
start = time()
df_isolation['is_anomaly'] = model.predict(X_processed)
inference_time = time() - start
df_isolation['is_anomaly'] = df_isolation['is_anomaly'].map({-1: 1, 1: 0})  # Convert to binary labels

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s finished


In [62]:
df_isolation

,estimated_traffic,grid_id,l1,l2,l3,l4,l5,lag_24h,lag_48h,datetime,...,day_of_month,week_of_year,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,anomaly_score,is_anomaly
0,65,10_1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,0.009301,0
1,33995,10_10,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,-0.001659,1
2,2730,10_11,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,0.009706,0
3,780,10_12,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,0.009735,0
4,260,10_13,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,0.008992,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173451,195,5_13,130.0,65.0,65.0,195.0,0.0,455.0,130.0,2008-02-08 17:30:00,...,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,0.012126,0
173452,260,19_19,585.0,455.0,520.0,650.0,585.0,585.0,585.0,2008-02-08 17:30:00,...,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,0.007944,0
173453,650,19_20,780.0,780.0,780.0,650.0,780.0,910.0,975.0,2008-02-08 17:30:00,...,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,0.001447,0
173454,0,3_14,0.0,0.0,0.0,0.0,65.0,0.0,0.0,2008-02-08 17:30:00,...,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,0.014787,0


In [63]:
display(df_isolation["is_anomaly"].sum())
display(float(df_isolation['is_anomaly'].mean()))

8673
0.05000115303016327


In [64]:
display(f"Isolation Training duration : {train_time}")
display(f"Isolation Inference duration : {inference_time}")

Isolation Training duration : 10.268640756607056
Isolation Inference duration : 6.823688745498657


In [65]:
delta_columns = ["l1", "l2", "l3", "l4", "l5", "lag_24h", "lag_48h", "lag_72h"]

for column in delta_columns:    
    df_isolation[f"delta_{column}"] = np.where(
        df_isolation[column] != -1,
        df_isolation["estimated_traffic"] - df_isolation[column],
        df_isolation["estimated_traffic"]
    )
df_isolation['z_traffic'] = (df_isolation['estimated_traffic'] - df_isolation['estimated_traffic'].mean()) / df_isolation['estimated_traffic'].std()

##
hourly_anomalies = df_isolation[df_isolation['is_anomaly'] == 1].groupby('hour').size().reset_index(name="hourly_anomalies")
hourly_anomalies["hourly_count"] = df_isolation.groupby('hour').size()
hourly_anomalies["hourly_anomalies_ratio"] = hourly_anomalies["hourly_anomalies"] / hourly_anomalies["hourly_count"]
##
day_of_week_anomalies = df_isolation[df_isolation['is_anomaly'] == 1].groupby('day_of_week').size().reset_index(name="day_of_week_anomalies")
day_of_week_anomalies["day_of_week_count"] = df_isolation.groupby('day_of_week').size()
day_of_week_anomalies["day_of_week_anomalies_ratio"] = day_of_week_anomalies["day_of_week_anomalies"] / day_of_week_anomalies["day_of_week_count"]

##
weekend_vs_weekday = df_isolation[df_isolation["is_anomaly"] == 1].groupby('is_weekend').size().reset_index(name="is_weekend_anomaly")
weekend_vs_weekday["is_weekend_count"] = df_isolation.groupby("is_weekend").size()
weekend_vs_weekday["is_weekend_ratio"] = weekend_vs_weekday["is_weekend_anomaly"] / weekend_vs_weekday["is_weekend_count"]
##
hourly_anomalies.to_csv("hourly_anomalies_isolation.csv", index=False)
day_of_week_anomalies.to_csv("day_of_week_anomalies_isolation.csv", index=False)
weekend_vs_weekday.to_csv("weekend_weekday_anomalies_isolation.csv", index=False)
##
display(hourly_anomalies)
display(day_of_week_anomalies)
display(weekend_vs_weekday)

,hour,hourly_anomalies,hourly_count,hourly_anomalies_ratio
0,0,367,7032,0.052190
1,1,304,7032,0.043231
2,2,236,6739,0.035020
3,3,206,7032,0.029295
4,4,173,7032,0.024602
5,5,121,7032,0.017207
6,6,89,7032,0.012656
7,7,119,7032,0.016923
8,8,179,7032,0.025455
9,9,274,7032,0.038965


,day_of_week,day_of_week_anomalies,day_of_week_count,day_of_week_anomalies_ratio
0,0,1870,28128,0.066482
1,1,1431,28128,0.050875
2,2,1181,28128,0.041987
3,3,781,28128,0.027766
4,4,560,20510,0.027304
5,5,1189,12306,0.096620
6,6,1661,28128,0.059051


,is_weekend,is_weekend_anomaly,is_weekend_count,is_weekend_ratio
0,0,5823,133022,0.043775
1,1,2850,40434,0.070485


In [ ]:
total_counts = df_isolation.groupby('grid_id').size().reset_index(name='total_count')
anomaly_counts = df_isolation[df_isolation['is_anomaly'] == 1].groupby('grid_id').size().reset_index(name='anomaly_count')

display(anomaly_counts)

merged = pd.merge(total_counts, anomaly_counts, on='grid_id', how='inner')
merged['anomaly_ratio'] = merged['anomaly_count'] / merged['total_count']
merged.to_csv("anomaly_ratio_per_grid_isolation.csv", index=False)
merged.sort_values("anomaly_ratio", ascending=False)

In [69]:
df_isolation.to_csv("anomaly_isolation_tree.csv", index=False)

# ONE CLASS SVM

In [5]:
df_svm = traffic_data_if.copy()

In [ ]:
features = ['estimated_traffic', 'grid_id', 'l1', 'l2', 'l3', 'l4', 'l5', 'lag_24h', 'lag_48h',
       'lag_72h', 'hour', 'day_of_week', 'is_weekend', 'quarter',
       'day_of_month', 'week_of_year', 'hour_sin', 'hour_cos',
       'day_of_week_sin', 'day_of_week_cos', 'day_of_month_sin',
       'day_of_month_cos'
]

X = df_svm[features]

start = time()

# Train One Class SVM Forest model
model = OneClassSVM(kernel='rbf', gamma='auto', nu=0.05)
model.fit(X)

train_time = time() - start

with open('one_class_svm.pkl', 'wb') as f:
   pickle.dump(model, f)
# Add predictions to dataframe (optional)
df_svm['anomaly_score'] = model.decision_function(X)
start = time()
df_svm['is_anomaly'] = model.predict(X)
inference_time = time() - start
df_svm['is_anomaly'] = df_svm['is_anomaly'].map({-1: 1, 1: 0})  # Convert to binary labels

In [ ]:
display(f"SVM Training time : {train_time}")
display(f"SVM Inference time : {inference_time}")

In [ ]:
delta_columns = ["l1", "l2", "l3", "l4", "l5", "lag_24h", "lag_48h", "lag_72h"]

for column in delta_columns:    
    df_svm[f"delta_{column}"] = np.where(
        df_svm[column] != -1,
        df_svm["estimated_traffic"] - df_svm[column],
        df_svm["estimated_traffic"]
    )
df_svm['z_traffic'] = (df_svm['estimated_traffic'] - df_svm['estimated_traffic'].mean()) / df_svm['estimated_traffic'].std()
##
hourly_anomalies = df_svm[df_svm['is_anomaly'] == 1].groupby('hour').size().reset_index(name="hourly_anomalies")
hourly_anomalies["hourly_count"] = df_svm.groupby('hour').size()
hourly_anomalies["hourly_anomalies_ratio"] = hourly_anomalies["hourly_anomalies"] / hourly_anomalies["hourly_count"]
##
day_of_week_anomalies = df_svm[df_svm['is_anomaly'] == 1].groupby('day_of_week').size().reset_index(name="day_of_week_anomalies")
day_of_week_anomalies["day_of_week_count"] = df_svm.groupby('day_of_week').size()
day_of_week_anomalies["day_of_week_anomalies_ratio"] = day_of_week_anomalies["day_of_week_anomalies"] / day_of_week_anomalies["day_of_week_count"]

##
weekend_vs_weekday = df_svm[df_svm["is_anomaly"] == 1].groupby('is_weekend').size().reset_index(name="is_weekend_anomaly")
weekend_vs_weekday["is_weekend_count"] = df_svm.groupby("is_weekend").size()
weekend_vs_weekday["is_weekend_ratio"] = weekend_vs_weekday["is_weekend_anomaly"] / weekend_vs_weekday["is_weekend_count"]
##

hourly_anomalies.to_csv("hourly_anomalies_svm.csv", index=False)
day_of_week_anomalies.to_csv("day_of_week_anomalies_svm.csv", index=False)
weekend_vs_weekday.to_csv("weekend_weekday_anomalies_svm.csv", index=False)
display(hourly_anomalies)
display(day_of_week_anomalies)
display(weekend_vs_weekday)

In [ ]:
total_counts = df_svm.groupby('grid_id').size().reset_index(name='total_count')
anomaly_counts = df_svm[df_svm['is_anomaly'] == 1].groupby('grid_id').size().reset_index(name='anomaly_count')

display(anomaly_counts)

merged_svm = pd.merge(total_counts, anomaly_counts, on='grid_id', how='inner')
merged_svm['anomaly_ratio'] = merged_svm['anomaly_count'] / merged_svm['total_count']
merged_svm.to_csv("anomaly_ratio_per_grid_svm.csv", index=False)
merged_svm.sort_values("anomaly_ratio", ascending=False)

In [2]:
df_svm.to_csv("one_class_svm_anomaly.csv", index=False)

NameError: name 'df_svm' is not defined

# LOCAL OUTLIER FACTOR

In [7]:
df_lof = traffic_data_if.copy()

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

features = ['estimated_traffic', 'grid_id', 'l1', 'l2', 'l3', 'l4', 'l5', 'lag_24h', 'lag_48h',
       'lag_72h', 'hour', 'day_of_week', 'is_weekend', 'quarter',
       'day_of_month', 'week_of_year', 'hour_sin', 'hour_cos',
       'day_of_week_sin', 'day_of_week_cos', 'day_of_month_sin',
       'day_of_month_cos'
]

X = df_lof[features]

start = time()

clf = LocalOutlierFactor(n_neighbors=100, contamination=0.05)

train_time_lof = time() - start

with open('lof_model.pkl', 'wb') as f:
    pickle.dump(clf, f)

# Add predictions to dataframe (optional)
start = time()
df_lof['is_anomaly'] = clf.fit_predict(X)
inference_time_lof = time() - start
df_lof['is_anomaly'] = df_lof['is_anomaly'].map({-1: 1, 1: 0})  # Convert to binary labels

In [10]:
df_lof

,estimated_traffic,grid_id,l1,l2,l3,l4,l5,lag_24h,lag_48h,datetime,...,quarter,day_of_month,week_of_year,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,is_anomaly
0,65,10_1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,1,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,1
1,33995,10_10,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,1,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,1
2,2730,10_11,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,1,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,1
3,780,10_12,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,1,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,1
4,260,10_13,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,1,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173451,195,5_13,130.0,65.0,65.0,195.0,0.0,455.0,130.0,2008-02-08 17:30:00,...,1,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,0
173452,260,19_19,585.0,455.0,520.0,650.0,585.0,585.0,585.0,2008-02-08 17:30:00,...,1,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,0
173453,650,19_20,780.0,780.0,780.0,650.0,780.0,910.0,975.0,2008-02-08 17:30:00,...,1,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,0
173454,0,3_14,0.0,0.0,0.0,0.0,65.0,0.0,0.0,2008-02-08 17:30:00,...,1,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,1


In [11]:
display(train_time_lof)
display(inference_time_lof)

0.0003018379211425781
122.12959933280945


In [12]:
display(df_lof["is_anomaly"].sum())
display(float(df_lof['is_anomaly'].mean()))

8673
0.05000115303016327


In [13]:
delta_columns = ["l1", "l2", "l3", "l4", "l5", "lag_24h", "lag_48h", "lag_72h"]

for column in delta_columns:    
    df_lof[f"delta_{column}"] = np.where(
        df_lof[column] != -1,
        df_lof["estimated_traffic"] - df_lof[column],
        df_lof["estimated_traffic"]
    )
df_lof['z_traffic'] = (df_lof['estimated_traffic'] - df_lof['estimated_traffic'].mean()) / df_lof['estimated_traffic'].std()
##
hourly_anomalies = df_lof[df_lof['is_anomaly'] == 1].groupby('hour').size().reset_index(name="hourly_anomalies")
hourly_anomalies["hourly_count"] = df_lof.groupby('hour').size()
hourly_anomalies["hourly_anomalies_ratio"] = hourly_anomalies["hourly_anomalies"] / hourly_anomalies["hourly_count"]
##
day_of_week_anomalies = df_lof[df_lof['is_anomaly'] == 1].groupby('day_of_week').size().reset_index(name="day_of_week_anomalies")
day_of_week_anomalies["day_of_week_count"] = df_lof.groupby('day_of_week').size()
day_of_week_anomalies["day_of_week_anomalies_ratio"] = day_of_week_anomalies["day_of_week_anomalies"] / day_of_week_anomalies["day_of_week_count"]
##
weekend_vs_weekday = df_lof[df_lof["is_anomaly"] == 1].groupby('is_weekend').size().reset_index(name="is_weekend_anomaly")
weekend_vs_weekday["is_weekend_count"] = df_lof.groupby("is_weekend").size()
weekend_vs_weekday["is_weekend_ratio"] = weekend_vs_weekday["is_weekend_anomaly"] / weekend_vs_weekday["is_weekend_count"]
##
hourly_anomalies.to_csv("hourly_anomalies_lof.csv", index=False)
day_of_week_anomalies.to_csv("day_of_week_anomalies_lof.csv", index=False)
weekend_vs_weekday.to_csv("weekend_weekday_anomalies_lof.csv", index=False)
display(hourly_anomalies)
display(day_of_week_anomalies)
display(weekend_vs_weekday)

    hour  hourly_anomalies  hourly_count  hourly_anomalies_ratio
0      0               233          7032                0.033134
1      1               204          7032                0.029010
2      2               900          6739                0.133551
3      3               580          7032                0.082480
4      4               184          7032                0.026166
5      5               174          7032                0.024744
6      6               117          7032                0.016638
7      7               168          7032                0.023891
8      8               198          7032                0.028157
9      9               330          7032                0.046928
10    10               360          7032                0.051195
11    11               308          7032                0.043800
12    12               313          7032                0.044511
13    13               596          7618                0.078236
14    14               69

In [14]:
total_counts = df_lof.groupby('grid_id').size().reset_index(name='total_count')
anomaly_counts = df_lof[df_lof['is_anomaly'] == 1].groupby('grid_id').size().reset_index(name='anomaly_count')

display(anomaly_counts)

merged_lof = pd.merge(total_counts, anomaly_counts, on='grid_id', how='inner')
merged_lof['anomaly_ratio'] = merged_lof['anomaly_count'] / merged_lof['total_count']
merged_lof.to_csv("anomaly_ratio_per_grid_lof.csv", index=False)
merged_lof.sort_values("anomaly_ratio", ascending=False)

    grid_id  anomaly_count
0      10_1             57
1     10_10             37
2     10_11             14
3     10_12              8
4     10_13              3
..      ...            ...
276     9_5             19
277     9_6             19
278     9_7             22
279     9_8             18
280     9_9             43

[281 rows x 2 columns]


,grid_id,total_count,anomaly_count,anomaly_ratio
200,3_11,592,256,0.432432
199,3_10,592,238,0.402027
201,3_14,592,204,0.344595
27,11_9,592,192,0.324324
168,22_19,592,185,0.312500
...,...,...,...,...
45,13_14,592,1,0.001689
272,9_15,592,1,0.001689
8,10_4,592,1,0.001689
231,6_14,592,1,0.001689


In [13]:
df_lof.to_csv("LOF/anomaly_lof.csv", index=False)

# ELLIPTIC ENVELOPE

In [17]:
df_elliptic = traffic_data_if.copy()

In [19]:
from sklearn.covariance import EllipticEnvelope

features = ['estimated_traffic', 'grid_id', 'l1', 'l2', 'l3', 'l4', 'l5', 'lag_24h', 'lag_48h',
       'lag_72h', 'hour', 'day_of_week', 'is_weekend', 'quarter',
       'day_of_month', 'week_of_year', 'hour_sin', 'hour_cos',
       'day_of_week_sin', 'day_of_week_cos', 'day_of_month_sin',
       'day_of_month_cos'
]

X = df_elliptic[features]

start = time()

model = EllipticEnvelope(contamination=0.05)
model.fit(X)

train_time_elliptic = time() - start

with open('elliptic_enveloppe_model.pkl', 'wb') as f:
    pickle.dump(model, f)

df_elliptic['anomaly_score'] = model.decision_function(X)
start = time()
df_elliptic['is_anomaly'] = model.predict(X)
inference_time_elliptic = time() - start
df_elliptic['is_anomaly'] = df_elliptic['is_anomaly'].map({-1: 1, 1: 0})  # Convert to binary labels


/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/sklearn/covariance/_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


In [20]:
df_elliptic

,estimated_traffic,grid_id,l1,l2,l3,l4,l5,lag_24h,lag_48h,datetime,...,day_of_month,week_of_year,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,anomaly_score,is_anomaly
0,65,10_1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,4078.720882,0
1,33995,10_10,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,-143523.337499,1
2,2730,10_11,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,3116.536320,0
3,780,10_12,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,3998.519502,0
4,260,10_13,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,4069.836236,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173451,195,5_13,130.0,65.0,65.0,195.0,0.0,455.0,130.0,2008-02-08 17:30:00,...,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,4570.230618,0
173452,260,19_19,585.0,455.0,520.0,650.0,585.0,585.0,585.0,2008-02-08 17:30:00,...,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,4560.150402,0
173453,650,19_20,780.0,780.0,780.0,650.0,780.0,910.0,975.0,2008-02-08 17:30:00,...,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,4558.379778,0
173454,0,3_14,0.0,0.0,0.0,0.0,65.0,0.0,0.0,2008-02-08 17:30:00,...,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,4577.230887,0


In [24]:
display(df_elliptic["is_anomaly"].sum())
display(float(df_elliptic['is_anomaly'].mean()))

8673
0.05000115303016327


In [25]:
display(f"Training duration : {train_time_elliptic}")
display(f"Inference duration : {inference_time_elliptic}")

Training duration : 50.382025480270386
Inference duration : 0.3309152126312256


In [26]:
total_counts = df_elliptic.groupby('grid_id').size().reset_index(name='total_count')
anomaly_counts = df_elliptic[df_elliptic['is_anomaly'] == 1].groupby('grid_id').size().reset_index(name='anomaly_count')

display(anomaly_counts)


merged_elliptic = pd.merge(total_counts, anomaly_counts, on='grid_id', how='inner')
merged_elliptic['anomaly_ratio'] = merged_elliptic['anomaly_count'] / merged_lof['total_count']
merged_elliptic.to_csv("anomaly_ratio_per_grid_elliptic.csv", index=False)
merged_elliptic.sort_values("anomaly_ratio", ascending=False)

   grid_id  anomaly_count
0    10_10            504
1     10_7            194
2     10_8            558
3     10_9            521
4    11_10            554
5    11_11              1
6     11_7            124
7     11_8            588
8     11_9            592
9    12_10            592
10   12_11            166
11    12_8            470
12    12_9            582
13   13_11            160
14    13_9            102
15   14_11             24
16   14_12            449
17   25_13             11
18    8_10            141
19     8_8            328
20     8_9            417
21    9_10            479
22    9_11             67
23     9_5              2
24     9_7              4
25     9_8            542
26     9_9            501


,grid_id,total_count,anomaly_count,anomaly_ratio
9,12_10,592,592,1.000000
8,11_9,592,592,1.000000
7,11_8,592,588,0.993243
12,12_9,592,582,0.983108
2,10_8,592,558,0.942568
4,11_10,592,554,0.935811
25,9_8,592,542,0.915541
3,10_9,592,521,0.880068
0,10_10,592,504,0.851351
26,9_9,592,501,0.846284


In [27]:
delta_columns = ["l1", "l2", "l3", "l4", "l5", "lag_24h", "lag_48h", "lag_72h"]
for column in delta_columns:    
    df_elliptic[f"delta_{column}"] = np.where(
        df_elliptic[column] != -1,
        df_elliptic["estimated_traffic"] - df_elliptic[column],
        df_elliptic["estimated_traffic"]
    )

In [159]:
df_elliptic

,estimated_traffic,grid_id,l1,l2,l3,l4,l5,lag_24h,lag_48h,lag_72h,...,anomaly_score,is_anomaly,delta_l1,delta_l2,delta_l3,delta_l4,delta_l5,delta_lag_24h,delta_lag_48h,delta_lag_72h
0,65,10_1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,103375.031218,0,65.0,65.0,65.0,65.0,65.0,65.0,65.0,65.0
1,33995,10_10,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-130757.700782,1,33995.0,33995.0,33995.0,33995.0,33995.0,33995.0,33995.0,33995.0
2,2730,10_11,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,94687.137179,0,2730.0,2730.0,2730.0,2730.0,2730.0,2730.0,2730.0,2730.0
3,780,10_12,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,97670.272709,0,780.0,780.0,780.0,780.0,780.0,780.0,780.0,780.0
4,260,10_13,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,103370.647180,0,260.0,260.0,260.0,260.0,260.0,260.0,260.0,260.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173451,195,5_13,130.0,65.0,65.0,195.0,0.0,455.0,130.0,65.0,...,108713.348410,0,65.0,130.0,130.0,0.0,195.0,-260.0,65.0,130.0
173452,260,19_19,585.0,455.0,520.0,650.0,585.0,585.0,585.0,455.0,...,108711.696611,0,-325.0,-195.0,-260.0,-390.0,-325.0,-325.0,-325.0,-195.0
173453,650,19_20,780.0,780.0,780.0,650.0,780.0,910.0,975.0,390.0,...,108711.458983,0,-130.0,-130.0,-130.0,0.0,-130.0,-260.0,-325.0,260.0
173454,0,3_14,0.0,0.0,0.0,0.0,65.0,0.0,0.0,0.0,...,108732.842678,0,0.0,0.0,0.0,0.0,-65.0,0.0,0.0,0.0


In [28]:
df_elliptic['z_traffic'] = (df_elliptic['estimated_traffic'] - df_elliptic['estimated_traffic'].mean()) / df_elliptic['estimated_traffic'].std()
##
hourly_anomalies = df_elliptic[df_elliptic['is_anomaly'] == 1].groupby('hour').size().reset_index(name="hourly_anomalies")
hourly_anomalies["hourly_count"] = df_elliptic.groupby('hour').size()
hourly_anomalies["hourly_anomalies_ratio"] = hourly_anomalies["hourly_anomalies"] / hourly_anomalies["hourly_count"]
##
day_of_week_anomalies = df_elliptic[df_elliptic['is_anomaly'] == 1].groupby('day_of_week').size().reset_index(name="day_of_week_anomalies")
day_of_week_anomalies["day_of_week_count"] = df_elliptic.groupby('day_of_week').size()
day_of_week_anomalies["day_of_week_anomalies_ratio"] = day_of_week_anomalies["day_of_week_anomalies"] / day_of_week_anomalies["day_of_week_count"]

##
weekend_vs_weekday = df_elliptic[df_elliptic["is_anomaly"] == 1].groupby('is_weekend').size().reset_index(name="is_weekend_anomaly")
weekend_vs_weekday["is_weekend_count"] = df_elliptic.groupby("is_weekend").size()
weekend_vs_weekday["is_weekend_ratio"] = weekend_vs_weekday["is_weekend_anomaly"] / weekend_vs_weekday["is_weekend_count"]
##
hourly_anomalies.to_csv("hourly_anomalies_elliptic.csv", index=False)
day_of_week_anomalies.to_csv("day_of_week_anomalies_elliptic.csv", index=False)
weekend_vs_weekday.to_csv("weekend_weekday_anomalies_elliptic.csv", index=False)
display(hourly_anomalies)
display(day_of_week_anomalies)
display(weekend_vs_weekday)

    hour  hourly_anomalies  hourly_count  hourly_anomalies_ratio
0      0               377          7032                0.053612
1      1               354          7032                0.050341
2      2               343          6739                0.050898
3      3               281          7032                0.039960
4      4               209          7032                0.029721
5      5               165          7032                0.023464
6      6               140          7032                0.019909
7      7               189          7032                0.026877
8      8               289          7032                0.041098
9      9               388          7032                0.055176
10    10               422          7032                0.060011
11    11               421          7032                0.059869
12    12               424          7032                0.060296
13    13               452          7618                0.059333
14    14               48

In [29]:
df_elliptic.to_csv("anomaly_elliptic.csv", index=False)

# HBOS Clustering

In [15]:
df_hbos = traffic_data_if.copy()

In [31]:
from pyod.models.hbos import HBOS

features = ['estimated_traffic', 'grid_id', 'l1', 'l2', 'l3', 'l4', 'l5', 'lag_24h', 'lag_48h',
       'lag_72h', 'hour', 'day_of_week', 'is_weekend', 'quarter',
       'day_of_month', 'week_of_year', 'hour_sin', 'hour_cos',
       'day_of_week_sin', 'day_of_week_cos', 'day_of_month_sin',
       'day_of_month_cos'
]

X = df_hbos[features]

start = time()

model = HBOS()
model.fit(X)

train_time_hbos = time() - start

with open('hbos_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Add predictions to dataframe (optional)
df_hbos['anomaly_score'] = model.decision_function(X)
start = time()
df_hbos['is_anomaly'] = model.predict(X)
inference_time_hbos = time() - start
display(df_hbos)
df_hbos['is_anomaly'] = df_hbos['is_anomaly'].map({0: 1, 1: 0})  # Convert to binary labels

,estimated_traffic,grid_id,l1,l2,l3,l4,l5,lag_24h,lag_48h,datetime,...,day_of_month,week_of_year,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,anomaly_score,is_anomaly
0,65,10_1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,41.050084,1
1,33995,10_10,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,41.075214,1
2,2730,10_11,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,41.073297,1
3,780,10_12,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,41.073297,1
4,260,10_13,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,2,5,-0.258819,-0.965926,-0.974928,-0.222521,0.394356,0.918958,41.073297,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173451,195,5_13,130.0,65.0,65.0,195.0,0.0,455.0,130.0,2008-02-08 17:30:00,...,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,34.857000,0
173452,260,19_19,585.0,455.0,520.0,650.0,585.0,585.0,585.0,2008-02-08 17:30:00,...,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,34.856056,0
173453,650,19_20,780.0,780.0,780.0,650.0,780.0,910.0,975.0,2008-02-08 17:30:00,...,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,34.856056,0
173454,0,3_14,0.0,0.0,0.0,0.0,65.0,0.0,0.0,2008-02-08 17:30:00,...,8,6,-0.965926,-0.258819,-0.433884,-0.900969,0.998717,-0.050649,34.857000,0


In [32]:
display(df_hbos["is_anomaly"].sum())
display(float(df_hbos['is_anomaly'].mean()))

156110
0.8999976939396734


In [33]:
display(f"Training duration : {train_time_hbos}")
display(f"Inference duration : {inference_time_hbos}")

Training duration : 2.7239534854888916
Inference duration : 0.29566335678100586


In [34]:
delta_columns = ["l1", "l2", "l3", "l4", "l5", "lag_24h", "lag_48h", "lag_72h"]

for column in delta_columns:    
    df_hbos[f"delta_{column}"] = np.where(
        df_hbos[column] != -1,
        df_hbos["estimated_traffic"] - df_hbos[column],
        df_hbos["estimated_traffic"]
    )

In [35]:
df_hbos

,estimated_traffic,grid_id,l1,l2,l3,l4,l5,lag_24h,lag_48h,datetime,...,anomaly_score,is_anomaly,delta_l1,delta_l2,delta_l3,delta_l4,delta_l5,delta_lag_24h,delta_lag_48h,delta_lag_72h
0,65,10_1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,41.050084,0,65.0,65.0,65.0,65.0,65.0,65.0,65.0,65.0
1,33995,10_10,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,41.075214,0,33995.0,33995.0,33995.0,33995.0,33995.0,33995.0,33995.0,33995.0
2,2730,10_11,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,41.073297,0,2730.0,2730.0,2730.0,2730.0,2730.0,2730.0,2730.0,2730.0
3,780,10_12,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,41.073297,0,780.0,780.0,780.0,780.0,780.0,780.0,780.0,780.0
4,260,10_13,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2008-02-02 13:30:00,...,41.073297,0,260.0,260.0,260.0,260.0,260.0,260.0,260.0,260.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173451,195,5_13,130.0,65.0,65.0,195.0,0.0,455.0,130.0,2008-02-08 17:30:00,...,34.857000,1,65.0,130.0,130.0,0.0,195.0,-260.0,65.0,130.0
173452,260,19_19,585.0,455.0,520.0,650.0,585.0,585.0,585.0,2008-02-08 17:30:00,...,34.856056,1,-325.0,-195.0,-260.0,-390.0,-325.0,-325.0,-325.0,-195.0
173453,650,19_20,780.0,780.0,780.0,650.0,780.0,910.0,975.0,2008-02-08 17:30:00,...,34.856056,1,-130.0,-130.0,-130.0,0.0,-130.0,-260.0,-325.0,260.0
173454,0,3_14,0.0,0.0,0.0,0.0,65.0,0.0,0.0,2008-02-08 17:30:00,...,34.857000,1,0.0,0.0,0.0,0.0,-65.0,0.0,0.0,0.0


In [36]:
df_hbos['z_traffic'] = (df_hbos['estimated_traffic'] - df_hbos['estimated_traffic'].mean()) / df_hbos['estimated_traffic'].std()
##
hourly_anomalies = df_hbos[df_hbos['is_anomaly'] == 1].groupby('hour').size().reset_index(name="hourly_anomalies")
hourly_anomalies["hourly_count"] = df_hbos.groupby('hour').size()
hourly_anomalies["hourly_anomalies_ratio"] = hourly_anomalies["hourly_anomalies"] / hourly_anomalies["hourly_count"]
##
day_of_week_anomalies = df_hbos[df_hbos['is_anomaly'] == 1].groupby('day_of_week').size().reset_index(name="day_of_week_anomalies")
day_of_week_anomalies["day_of_week_count"] = df_hbos.groupby('day_of_week').size()
day_of_week_anomalies["day_of_week_anomalies_ratio"] = day_of_week_anomalies["day_of_week_anomalies"] / day_of_week_anomalies["day_of_week_count"]

##
weekend_vs_weekday = df_hbos[df_hbos["is_anomaly"] == 1].groupby('is_weekend').size().reset_index(name="is_weekend_anomaly")
weekend_vs_weekday["is_weekend_count"] = df_hbos.groupby("is_weekend").size()
weekend_vs_weekday["is_weekend_ratio"] = weekend_vs_weekday["is_weekend_anomaly"] / weekend_vs_weekday["is_weekend_count"]
##
hourly_anomalies.to_csv("hourly_anomalies_hbos.csv", index=False)
day_of_week_anomalies.to_csv("day_of_week_anomalies_hbos.csv", index=False)
weekend_vs_weekday.to_csv("weekend_weekday_anomalies_hbos.csv", index=False)
display(hourly_anomalies)
display(day_of_week_anomalies)
display(weekend_vs_weekday)

    hour  hourly_anomalies  hourly_count  hourly_anomalies_ratio
0      0              5860          7032                0.833333
1      1              7032          7032                1.000000
2      2              6739          6739                1.000000
3      3              5860          7032                0.833333
4      4              7032          7032                1.000000
5      5              7032          7032                1.000000
6      6              6680          7032                0.949943
7      7              7032          7032                1.000000
8      8              7032          7032                1.000000
9      9              5860          7032                0.833333
10    10              7032          7032                1.000000
11    11              7032          7032                1.000000
12    12              7032          7032                1.000000
13    13              7032          7618                0.923077
14    14              703

In [37]:
total_counts = df_hbos.groupby('grid_id').size().reset_index(name='total_count')
anomaly_counts = df_hbos[df_hbos['is_anomaly'] == 1].groupby('grid_id').size().reset_index(name='anomaly_count')

display(anomaly_counts)

merged = pd.merge(total_counts, anomaly_counts, on='grid_id', how='inner')
merged['anomaly_ratio'] = merged['anomaly_count'] / merged['total_count']
merged.to_csv("anomaly_ratio_per_grid_hbos.csv", index=False)
merged.sort_values("anomaly_ratio", ascending=False)

    grid_id  anomaly_count
0      10_1            534
1     10_10            530
2     10_11            534
3     10_12            534
4     10_13            534
..      ...            ...
288     9_5            534
289     9_6            534
290     9_7            534
291     9_8            534
292     9_9            534

[293 rows x 2 columns]


,grid_id,total_count,anomaly_count,anomaly_ratio
0,10_1,592,534,0.902027
200,2_20,592,534,0.902027
151,1_3,592,534,0.902027
152,1_4,592,534,0.902027
153,1_5,592,534,0.902027
...,...,...,...,...
142,19_19,592,530,0.895270
143,19_20,592,530,0.895270
169,21_19,592,530,0.895270
229,5_15,592,530,0.895270


In [38]:
df_hbos.to_csv("anomaly_hbos.csv", index=False)
